In [1]:
import db.mariadb as mariadb

con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

compressed_data = mariadb.get_every_nth_file_feature(con, 1)
uncompressed_data = mariadb.get_every_nth_file_feature_uncompressed(con, 1)

con.close()


In [2]:
import numpy as np
import json

compressed_features = np.array([json.loads(data[1]) for data  in compressed_data])
uncompressed_features = np.array([json.loads(data[1]) for data  in uncompressed_data])

print('compressed features shape: ' + str(compressed_features.shape))
print('uncompressed features shape: ' + str(uncompressed_features.shape))


compressed features shape: (14984, 32)
uncompressed features shape: (14984, 2048)


In [1]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=100)

compressed_neighbours = [value.tolist() for value in nn.fit(compressed_features).kneighbors()]
uncompressed_neighbours = [value.tolist() for value in nn.fit(uncompressed_features).kneighbors()]


ModuleNotFoundError: No module named 'sklearn'

In [4]:
con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

for idx, (image_id, _feature) in enumerate(compressed_data):
    
    neighbour_ids = [compressed_data[i][0] for i in compressed_neighbours[1][idx]]
    neighbour_distances = compressed_neighbours[0][idx]
    mariadb.write_neighbours(
        image_id, list(zip(neighbour_ids, neighbour_distances)), con
    )
    
for idx, (image_id, _feature) in enumerate(uncompressed_data):
    neighbour_ids = [uncompressed_data[i][0] for i in uncompressed_neighbours[1][idx]]
    neighbour_distances = uncompressed_neighbours[0][idx]
    
    mariadb.write_uncompressed_neighbours(
        image_id, list(zip(neighbour_ids, neighbour_distances)), con
    )
    
con.close()
